Импортируйте пандас и прочитайте данные с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.

In [4]:
import pandas as pd

In [8]:
pd.read_csv('data.csv', encoding='ISO-8859-1').head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [9]:
retail = pd.read_csv('data.csv', encoding='ISO-8859-1')
retail_columns = retail.columns

Проверьте, встречаются ли в данных повторяющиеся наблюдения. Если они есть, то удалите их из retail.

In [10]:
retail.drop_duplicates(inplace=True)

Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
<br>
Сколько всего транзакций отменили пользователи? 

In [11]:
retail.InvoiceNo.str.startswith('C').sum()

9251

Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0.

In [12]:
retail = retail.query('Quantity > 0')

Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).

In [13]:
germany_top = retail \
                    .query('Country == "Germany"') \
                    .groupby('CustomerID', as_index=False) \
                    .agg({'InvoiceNo':'nunique'})

germany_top = germany_top[germany_top.InvoiceNo > germany_top.InvoiceNo.quantile(q=0.80)].CustomerID

Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

In [14]:
top_retail_germany = retail.query('CustomerID in @germany_top')

Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? <br>
Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [16]:
top_retail_germany.StockCode.value_counts().head()

StockCode
POST     213
22326     52
22328     38
22423     34
20719     30
Name: count, dtype: int64

In [17]:
# 22326

Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [18]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [20]:
sorted_revenues = retail.groupby('InvoiceNo', as_index=False) \
                        .agg({'Revenue':'sum'}) \
                        .sort_values('Revenue', ascending=False) \
                        .head(5)
sorted_revenues.InvoiceNo[:5].str.cat(sep=', ')

'581483, 541431, 574941, 576365, 556444'